In [1]:
#import sys
#sys.path.append("/home/klar/Promotion/projects/nonlocal-assembly")

# Setting

Let $\Omega = B_1$ and $\Omega_I = B_{1+\delta}\setminus B_1$, and $\overline{\Omega} = \Omega \cup \Omega_I$.
We are given $n$ Ansatzfunctions
$$
\Phi = (\phi_1, \phi_2)  \in \mathcal{L}(\overline{\Omega})^2.
$$
in a discretization of a 2-D Grid.

### Configuration

In [2]:
from duffyconf import *
import meshio
import matplotlib.pyplot as plt
import numpy as np

In [3]:
mesh = meshio.read(DATA_PATH + mesh_name)

### Ansatz function

In [28]:
class V:
    def __init__(self, mesh):
        self.points = mesh.points[:,:2]
        self.N = self.points.shape[0]
        self.triangles = mesh.cells[1][1]
        
    def phi1(self, p):
        return 1 - p[0] - p[1]
    
    def phi2(self, p):
        return p[0]
    
    def phi3(self, p):
        return p[1]
    
    def __getitem__(self, k):
        phi = [self.phi1, self.phi2, self.phi3]
        #n, k = index
        return phi[k]
    
    def plot(self):
        plt.scatter(self.points[:,0], self.points[:,1], s=.1)
        plt.scatter(0,0)
        plt.show()
    
class T:
    def __init__(self, mesh):
        self.triangles = mesh.cells[1][1]
        self.points = mesh.points[:,:2]
        self.N = self.triangles.shape[0]
        self.tris = []
        for k in range(self.N):
            Vdx = self.triangles[k]
            a,b,c = np.array(self.points[Vdx])
            self.tris.append(Triangle(a,b,c))
    def __getitem__(self, k):
        return self.tris[k]
    
class Triangle:
    def __init__(self, a,b,c):
        self.M = np.array([b-a, c-a]).T
        self.det = np.abs(np.linalg.det(self.M))
        self.E = np.array([a,b,c])
        self.a = a
        self.b = b
        self.c = c
        
    def __call__(self, x):
        return self.M@x + self.a
  
def integrate(x, dy, Py, ker, tri):
    n = dy.shape[0]
    s = 0#np.zeros((2,2))
    for k in range(n):
        y = tri(Py[k])
        #print(x, y)
        #raise KeyboardInterrupt
        s += ker(x, y)*tri.det*dy[k]#*np.outer(x-y,x-y)
    return s

def kernel(x,y):
    z = x-y
    d = np.linalg.norm(z)
    if d>1e-6:
        return 1./d#**3
    else:
        print("Zero Division!")
        return 1.

### Integral

We set $\delta = 0.3$.

As warm up we simply integrate the circle weighted with the kernel for $x = (0,0)$. The analytic solution is obtained by.
$$
\sum_{A \in \mathcal{T}} \int_A \frac{1}{\|y\|} dy
$$
Where the analytic solution is given by

$$
\int_{B_{1+\delta}} \frac{1}{\|y\|} dy = 
\int_0^{1+\delta} \int_0^{2\pi} \frac{r}{r} d\phi dr = (1+\delta) 2 \pi. 
$$
as the determinant of the Jacobian is $r$ for $\phi: (r, \phi) \mapsto r (sin(\phi), cos(\phi))$ .

In [33]:
Ansatz = V(mesh)
Triangles = T(mesh)

x = np.zeros((2,))
integral = 0.
Det = np.zeros(Triangles.N)
for k in range(Triangles.N):
    tri = Triangles[k]
    integral += integrate(x, dy, Py, kernel, tri)
print("Integral\t", integral)
print("True value\t", 2*np.pi*(1+delta))

Integral	 8.151759178074103
True value	 8.168140899333462
